In [2]:
# importng modules

In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, date, timedelta

In [4]:
# settings

In [5]:
Faker.seed(42)
random.seed(42)
fake = Faker("en_NZ")  # NZ locale for names

NUM_PATIENTS = 350
NUM_DOCTORS = 25
NUM_VISITS = 12000

VISIT_START = datetime(2022, 1, 1).date()
VISIT_END = datetime(2024, 12, 31).date()

In [6]:
# blood gruops - distributed logically

In [7]:
blood_types = ["O+", "A+", "B+", "AB+", "O-", "A-", "B-", "AB-"]
blood_weights = [38, 34, 9, 3, 6, 6, 3, 1]  # realistic percentages

In [8]:
# Towns in Auckland 

In [9]:
coastal_towns = ["Takapuna", "Albany", "Silverdale", "Howick", "Glenfield"]
inland_towns = ["Henderson", "Manukau", "Papakura", "New Lynn", "Pukekohe"]
all_towns = coastal_towns + inland_towns

In [10]:
# specialities

In [11]:
specialties = [
    "General Practitioner", "Cardiologist", "Dermatologist", "Pediatrician",
    "Orthopedic Surgeon", "Neurologist", "Psychiatrist", "ENT Specialist",
    "Endocrinologist", "Pulmonologist"
]

In [12]:
# departments

In [13]:
departments = {
    "General Practitioner": "General Medicine",
    "Cardiologist": "Cardiology",
    "Dermatologist": "Dermatology",
    "Pediatrician": "Pediatrics",
    "Orthopedic Surgeon": "Orthopedics",
    "Neurologist": "Neurology",
    "Psychiatrist": "Psychiatry",
    "ENT Specialist": "ENT",
    "Endocrinologist": "Endocrinology",
    "Pulmonologist": "Pulmonology"
}

In [14]:
# seasonal symptoms and diagnosis

In [15]:
seasonal_data = {
    "Summer": {
        "months": [12, 1, 2],
        "symptoms": ["Skin Rash", "Heat Exhaustion", "Hay Fever", "Nausea"],
        "diagnoses": ["Eczema", "Allergic Rhinitis", "Food Poisoning"]
    },
    "Autumn": {
        "months": [3, 4, 5],
        "symptoms": ["Cough", "Fever", "Asthma", "Sneezing"],
        "diagnoses": ["Bronchitis", "Viral Infection", "Asthma Flare-up"]
    },
    "Winter": {
        "months": [6, 7, 8],
        "symptoms": ["Cough", "Fever", "Chest Pain", "Shortness of Breath"],
        "diagnoses": ["Influenza", "Pneumonia", "Angina"]
    },
    "Spring": {
        "months": [9, 10, 11],
        "symptoms": ["Hay Fever", "Asthma", "Skin Rash", "Sinus Pain"],
        "diagnoses": ["Allergic Rhinitis", "Asthma Flare-up", "Sinusitis"]
    }
}

In [16]:
# symptom to speciality mapping

In [17]:
symptom_specialty_map = {
    "Skin Rash": "Dermatologist",
    "Heat Exhaustion": "General Practitioner",
    "Hay Fever": "ENT Specialist",
    "Nausea": "General Practitioner",
    "Cough": "Pulmonologist",
    "Fever": "General Practitioner",
    "Asthma": "Pulmonologist",
    "Sneezing": "ENT Specialist",
    "Chest Pain": "Cardiologist",
    "Shortness of Breath": "Cardiologist",
    "Sinus Pain": "ENT Specialist"
}


In [18]:
#symptom to diagnosis mapping

In [19]:
symptom_to_diag = {
    "Skin Rash": ["Eczema", "Contact Dermatitis"],
    "Heat Exhaustion": ["Heat Stroke", "Dehydration"],
    "Hay Fever": ["Allergic Rhinitis"],
    "Nausea": ["Food Poisoning", "Gastroenteritis"],
    "Cough": ["Bronchitis", "Viral Infection"],
    "Fever": ["Viral Infection", "Influenza"],
    "Asthma": ["Asthma Flare-up"],
    "Sneezing": ["Allergic Rhinitis", "Common Cold"],
    "Chest Pain": ["Angina", "Myocardial Infarction"],
    "Shortness of Breath": ["Pneumonia", "Asthma Flare-up"],
    "Sinus Pain": ["Sinusitis"]
}

In [20]:
# Age based chronic condition

In [21]:
def assign_chronic_condition(age):
    if age <= 12:
        return random.choices(["Asthma", "None", "Diabetes (Type 1)"], weights=[50, 40, 10])[0]
    elif 13 <= age <= 19:
        return random.choices(["Asthma", "None", "Anxiety"], weights=[40, 40, 20])[0]
    elif 20 <= age <= 35:
        return random.choices(["None", "Asthma", "Anxiety"], weights=[50, 30, 20])[0]
    elif 36 <= age <= 59:
        return random.choices(["Hypertension", "Diabetes (Type 2)", "Arthritis", "None"], weights=[40, 30, 20, 10])[0]
    else:
        return random.choices(["Hypertension", "Arthritis", "Heart Disease", "Chronic Kidney Disease"], weights=[35, 30, 20, 15])[0]

In [22]:
# generating patients

In [23]:
patients = []
six_years_ago = date.today() - timedelta(days=6*365)

for pid in range(1, NUM_PATIENTS + 1):
    gender = random.choice(["Male", "Female"])
    town = random.choice(all_towns)
    town_category = "Coastal" if town in coastal_towns else "Inland"
    dob = fake.date_of_birth(minimum_age=1, maximum_age=90)
    # Age as of today (ok for chronic assignment)
    age = int((date.today() - dob).days // 365)
    # Registration date must be <= VISIT_END
    ##reg_date = fake.date_between(start_date=six_years_ago, end_date=VISIT_END) -- old logic

    #new logic--   # --- Ensure reg_date >= dob and <= VISIT_END ---
    reg_start = max(dob, six_years_ago)
    if reg_start > VISIT_END:
        reg_start = VISIT_END - timedelta(days=1)  # last possible reg date
    reg_date = fake.date_between(start_date=reg_start, end_date=VISIT_END)
    
    chronic = assign_chronic_condition(age)
    blood = random.choices(blood_types, weights=blood_weights, k=1)[0]
    insurance = random.choices(["Yes", "No"], weights=[7,3], k=1)[0]

    patients.append({
        "PatientID": pid,
        "Name": fake.name_male() if gender == "Male" else fake.name_female(),
        "Gender": gender,
        "DOB": dob,
        "Age": age,
        "City": "Auckland",
        "Town": town,
        "TownCategory": town_category,
        "Insurance": insurance,
        "BloodType": blood,
        "ChronicCondition": chronic,
        "RegistrationDate": reg_date
    })


In [24]:
patients_df = pd.DataFrame(patients)
patients_df.to_csv("aucklandpatients.csv", index=False)

In [25]:
# doctors

In [26]:
doctors = []
# create a pool of doctors by speciality so we can pick later
for did in range(1, NUM_DOCTORS + 1):
    speciality = random.choice(specialties)
    doctors.append({
        "DoctorID": did,
        "Name": "Dr."+fake.name(),
        "Speciality": speciality,
        "Department": departments[speciality],
        "ExperienceYears": random.randint(2, 35),
        "City": "Auckland",
        "Town": random.choice(all_towns)
    })


In [27]:
doctors_df = pd.DataFrame(doctors)
doctors_df.to_csv("aucklanddoctors.csv", index=False)

In [28]:
# build a mapping speciality to list of Doctors ids

In [29]:
doctors_by_speciality = {}
for doc in doctors:
    doctors_by_speciality.setdefault(doc["Speciality"], []).append(doc["DoctorID"])

In [30]:
doctors_by_speciality

{'Neurologist': [1, 3, 6, 19, 22],
 'Pulmonologist': [2, 8, 9, 10],
 'Endocrinologist': [4, 11, 18],
 'Orthopedic Surgeon': [5, 24, 25],
 'Psychiatrist': [7, 16, 20],
 'Pediatrician': [12],
 'Cardiologist': [13],
 'ENT Specialist': [14, 17, 21, 23],
 'Dermatologist': [15]}

In [31]:
# build a mapping department to doctor ids 

In [32]:
doctors_by_department = {}
for doc in doctors:
    doctors_by_department.setdefault(doc["Department"], []).append(doc["DoctorID"])

In [33]:
doctors_by_department

{'Neurology': [1, 3, 6, 19, 22],
 'Pulmonology': [2, 8, 9, 10],
 'Endocrinology': [4, 11, 18],
 'Orthopedics': [5, 24, 25],
 'Psychiatry': [7, 16, 20],
 'Pediatrics': [12],
 'Cardiology': [13],
 'ENT': [14, 17, 21, 23],
 'Dermatology': [15]}

In [34]:
# generating visits

In [35]:
visits = []

for _ in range(NUM_VISITS):
    # --- pick patient ---
    patient = random.choice(patients)
    patient_id = patient["PatientID"]
    patient_age = patient["Age"]
    patient_town = patient["Town"]
    town_category = patient["TownCategory"]
    reg_date = pd.to_datetime(patient["RegistrationDate"]).date()

    # ensure visit date >= registration date
    #if reg_date > VISIT_END:   -- old logic
        #continue  # skip impossible cases

    start_for_patient = max(reg_date, VISIT_START)
    if start_for_patient > VISIT_END:
        continue  # no valid visit window

    days_range = (VISIT_END - start_for_patient).days
    visit_date = start_for_patient if days_range <= 0 else start_for_patient + timedelta(days=random.randint(0, days_range))


    # --- season selection ---
    month = visit_date.month
    for sname, details in seasonal_data.items():
        if month in details["months"]:
            season = sname
            season_symptoms = details["symptoms"]
            season_diagnoses = details["diagnoses"]
            break

    # --- symptom selection ---
    symptom = random.choice(season_symptoms)

    # coastal bias: in summer coastal towns → ear/skin/heat
    if town_category == "Coastal" and season == "Summer":
        symptom = random.choices(
            ["Skin Rash", "Heat Exhaustion", "Ear Infection", symptom],
            weights=[30, 25, 20, 25],
            k=1
        )[0]

    # age bias: kids & seniors
    if patient_age <= 12:
        symptom = random.choices(
            ["Ear Infection", "Common Cold", "Asthma", symptom],
            weights=[30, 25, 20, 25],
            k=1
        )[0]
    elif patient_age >= 65 and season == "Winter":
        symptom = random.choices(
            ["Chest Pain", "Shortness of Breath", "Cough", symptom],
            weights=[30, 25, 25, 20],
            k=1
        )[0]

    # --- doctor & department ---
    specialty = symptom_specialty_map.get(symptom, "General Practitioner")
    department = departments.get(specialty, "General Medicine")

    eligible_doctors = doctors_by_speciality.get(specialty, [])
    if not eligible_doctors:
        eligible_doctors = doctors_by_department.get(department, [])
    if not eligible_doctors:
        eligible_doctors = [d["DoctorID"] for d in doctors]  # fallback all doctors

    doctor_id = random.choice(eligible_doctors)

    # --- diagnosis selection ---
    diag_candidates = symptom_to_diag.get(symptom, []) + season_diagnoses
    if patient_age <= 12:
        diag_candidates += ["Ear Infection", "Common Cold"]
    if patient_age >= 65:
        diag_candidates += ["Pneumonia", "Heart Disease"]

    diagnosis = random.choice(list(set(diag_candidates)))  # avoid duplicate list entries

    # --- visit type & billing ---
    visit_type = random.choices(
        ["Outpatient", "Emergency", "Inpatient"],
        weights=[70, 20, 10],
        k=1
    )[0]
    payment_method = random.choices(
        ["Insurance", "Self-pay", "ACC"],
        weights=[70, 20, 10],
        k=1
    )[0]
    billing_amount = {
        "Outpatient": round(random.uniform(50, 200), 2),
        "Emergency": round(random.uniform(200, 1500), 2),
        "Inpatient": round(random.uniform(500, 5000), 2)
    }[visit_type]

    # --- follow-up logic ---
    base_followup_prob = 0.18
    if diagnosis in ["Arthritis", "Heart Disease", "Pneumonia", "Bronchitis", "Asthma Flare-up"]:
        base_followup_prob += 0.25
    if patient["ChronicCondition"] != "None":
        base_followup_prob += 0.08

    follow_up_required = "Yes" if random.random() < base_followup_prob else "No"

    followup_doctor_id = None
    if follow_up_required == "Yes":
        if random.random() < 0.7:
            followup_doctor_id = doctor_id
        else:
            same_dept_doctors = doctors_by_department.get(department, [])
            other_docs = [d for d in same_dept_doctors if d != doctor_id]
            followup_doctor_id = random.choice(other_docs) if other_docs else doctor_id

    # --- append visit (no VisitID yet) ---
    visits.append({
        "PatientID": patient_id,
        "DoctorID": doctor_id,
        "FollowUpDoctorID": followup_doctor_id,
        "Town": patient_town,
        "TownCategory": town_category,
        "Symptom": symptom,
        "Diagnosis": diagnosis,
        "VisitDate": visit_date,
        "VisitType": visit_type,
        "BillingAmount": billing_amount,
        "InsuranceUsed": "Yes" if payment_method == "Insurance" else "No",
        "PaymentMethod": payment_method,
        "PrescriptionGiven": random.choices(["Yes", "No"], weights=[70, 30], k=1)[0],
        "FollowUpRequired": follow_up_required,
        "ReadmissionFlag": None,  # will calculate later
        "VisitDurationMinutes": random.randint(5, 240)
    })

# --- chronological VisitID assignment ---
visits.sort(key=lambda x: x["VisitDate"])
for idx, visit in enumerate(visits, start=1):
    visit["VisitID"] = idx

In [36]:
# Convert to DataFrame and compute ReadmissionFlag
visits_df = pd.DataFrame(visits)

# Convert VisitDate to datetime.date if needed
visits_df["VisitDate"] = pd.to_datetime(visits_df["VisitDate"]).dt.date

# Sort visits by PatientID and VisitDate
visits_df.sort_values(["PatientID", "VisitDate"], inplace=True)
visits_df.reset_index(drop=True, inplace=True)

# Compute ReadmissionFlag: a visit is readmission if previous visit for same patient exists and
# current_visit_date - previous_visit_date <= 30 days AND previous was NOT a scheduled follow-up.
readmission_flags = []
prev_dates = {}  # patient_id -> (prev_date, prev_followup_required)

for idx, row in visits_df.iterrows():
    pid = row["PatientID"]
    vdate = pd.to_datetime(row["VisitDate"]).date()
    prev = prev_dates.get(pid)
    flag = "No"
    if prev:
        prev_date, prev_followup = prev
        days_diff = (vdate - prev_date).days
        # if current visit within 30 days of previous visit and previous was not a scheduled follow-up -> readmission
        if days_diff <= 30 and (prev_followup == "No"):
            flag = "Yes"
    # update prev_dates using current visit
    prev_dates[pid] = (vdate, row["FollowUpRequired"])
    readmission_flags.append(flag)

visits_df["ReadmissionFlag"] = readmission_flags


In [37]:
visits_df.to_csv("aucklandvisits.csv", index=False)

In [38]:
print("Datasets generated: patients.csv, doctors.csv, visits.csv")
print("  - RegistrationDate capped <="," VISIT_END")
print("  - VisitDate always >= RegistrationDate")
print("  - Symptom -> Specialty -> Doctor mapping applied")
print("  - FollowUpDoctor assigned (same or same-department)")
print("  - ReadmissionFlag computed (unplanned revisit within 30 days)")
print("  - Age-based chronic conditions, seasonal & coastal/inland bias, blood type weights applied")

Datasets generated: patients.csv, doctors.csv, visits.csv
  - RegistrationDate capped <=  VISIT_END
  - VisitDate always >= RegistrationDate
  - Symptom -> Specialty -> Doctor mapping applied
  - FollowUpDoctor assigned (same or same-department)
  - ReadmissionFlag computed (unplanned revisit within 30 days)
  - Age-based chronic conditions, seasonal & coastal/inland bias, blood type weights applied


In [39]:
# Loading to ssms

In [40]:
import pyodbc

In [42]:
#AucklandCityHospital

In [43]:
server = 'LAPTOP-7PGQLN8V\SQLEXPRESS'
database = 'AucklandCityHospitalLatest'
driver = '{ODBC Driver 17 for SQL Server}'
conn_str = f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [44]:
df_aucklandpatients_to_sql=pd.read_csv("aucklandpatients.csv",parse_dates=['DOB','RegistrationDate'])

In [45]:
df_aucklandpatients_to_sql

,PatientID,Name,Gender,DOB,Age,City,Town,TownCategory,Insurance,BloodType,ChronicCondition,RegistrationDate
0,1,Peter Jensen,Male,2021-08-19,3,Auckland,Takapuna,Coastal,Yes,O+,NaN,2022-02-07
1,2,Annette Grant,Male,1953-08-06,72,Auckland,New Lynn,Inland,Yes,A+,Hypertension,2020-06-25
2,3,David Davey,Male,1938-08-30,87,Auckland,Howick,Coastal,Yes,O+,Arthritis,2020-05-29
3,4,Jessica Ferguson,Female,2010-12-28,14,Auckland,Howick,Coastal,No,O+,NaN,2021-04-20
4,5,Oliver Harrison,Male,1972-06-19,53,Auckland,Manukau,Inland,No,O+,Hypertension,2019-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...
345,346,Karen Priest,Female,2010-01-03,15,Auckland,Pukekohe,Inland,No,O+,NaN,2024-06-24
346,347,Anthony Kennedy,Female,1952-04-07,73,Auckland,Henderson,Inland,No,O+,Heart Disease,2024-10-25
347,348,Rebecca Edmonds,Male,1999-10-02,25,Auckland,Howick,Coastal,Yes,A+,Asthma,2024-01-31
348,349,Melanie Ferguson,Male,1944-08-23,81,Auckland,Silverdale,Coastal,Yes,B+,Heart Disease,2020-09-04


In [46]:
df_aucklanddoctors_to_sql=pd.read_csv("aucklanddoctors.csv")

In [47]:
# Replace NaN with None for all columns
df_aucklandpatients_to_sql = df_aucklandpatients_to_sql.where(pd.notnull(df_aucklandpatients_to_sql), None)

# Convert date columns to Python date objects
df_aucklandpatients_to_sql['DOB'] = df_aucklandpatients_to_sql['DOB'].apply(lambda x: x.date() if x else None)
df_aucklandpatients_to_sql['RegistrationDate'] = df_aucklandpatients_to_sql['RegistrationDate'].apply(lambda x: x.date() if x else None)


In [49]:
df_aucklanddoctors_to_sql

,DoctorID,Name,Speciality,Department,ExperienceYears,City,Town
0,1,Dr.Christine Corlett-Kelly,Neurologist,Neurology,12,Auckland,Manukau
1,2,Dr.Samuel Pope,Pulmonologist,Pulmonology,21,Auckland,Pukekohe
2,3,Dr.Christopher Marshall-Carson,Neurologist,Neurology,34,Auckland,New Lynn
3,4,Dr.Tracy Rowe,Endocrinologist,Endocrinology,33,Auckland,Pukekohe
4,5,Dr.Scott Neal-Bright,Orthopedic Surgeon,Orthopedics,32,Auckland,Takapuna
5,6,Dr.Emma Parsons,Neurologist,Neurology,23,Auckland,Albany
6,7,Dr.Laura Brooks-Evans,Psychiatrist,Psychiatry,21,Auckland,Takapuna
7,8,Dr.Charlotte Herd-Parkes,Pulmonologist,Pulmonology,32,Auckland,Glenfield
8,9,Dr.Heather Bowater-Jones,Pulmonologist,Pulmonology,16,Auckland,Takapuna
9,10,Dr.Benjamin Jarvis,Pulmonologist,Pulmonology,32,Auckland,Silverdale


In [52]:
def insert_patients(cursor,df):
    for _,row in df.iterrows():
        cursor.execute("""
            INSERT INTO aucklandpatientslatest(PatientID,Name,Gender,DOB,Age,City,Town,TownCategory,Insurance,BloodType,ChronicCondition,RegistrationDate)
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?) 
            """,row.PatientID
               ,row.Name
               ,row.Gender
               ,row.DOB  
               ,row.Age
               ,row.City
               ,row.Town
               ,row.TownCategory
               ,row.Insurance
               ,row.BloodType
               ,row.ChronicCondition
               ,row.RegistrationDate
                      )
        conn.commit()

In [53]:
insert_patients(cursor,df_aucklandpatients_to_sql)

In [54]:
def insert_doctors(cursor,df):
    for _,row in df.iterrows():
        cursor.execute("""
            INSERT INTO aucklanddoctorslatest(DoctorID,Name,Speciality,Department,ExperienceYears,City,Town)
            VALUES(?,?,?,?,?,?,?) 
            """,row.DoctorID,row.Name,row.Speciality,row.Department,row.ExperienceYears,row.City,row.Town)
        conn.commit()

In [55]:
insert_doctors(cursor,df_aucklanddoctors_to_sql)

In [56]:
def insert_visits(cursor,df):
    for _,row in df.iterrows():
        cursor.execute("""
            INSERT INTO aucklandvisitslatest(
                PatientID,
                DoctorID,
                FollowUpDoctorID,
                Town,
                TownCategory,
                Symptom,
                Diagnosis,
                VisitDate,
                VisitType,
                BillingAmount,
                InsuranceUsed,
                PaymentMethod,
                PrescriptionGiven,
                FollowUpRequired,
                ReadmissionFlag,
                VisitDurationMinutes,
                VisitID )
                VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) 
            """,row.PatientID,
                row.DoctorID,
                row.FollowUpDoctorID,
                row.Town,
                row.TownCategory,
                row.Symptom,
                row.Diagnosis,
                row.VisitDate,
                row.VisitType,
                row.BillingAmount,
                row.InsuranceUsed,
                row.PaymentMethod,
                row.PrescriptionGiven,
                row.FollowUpRequired,
                row.ReadmissionFlag,
                row.VisitDurationMinutes,
                row.VisitID
                )
        conn.commit()

In [58]:
df_aucklandvisits_to_sql = pd.read_csv("aucklandvisits.csv")

In [59]:
df_aucklandvisits_to_sql

,PatientID,DoctorID,FollowUpDoctorID,Town,TownCategory,Symptom,Diagnosis,VisitDate,VisitType,BillingAmount,InsuranceUsed,PaymentMethod,PrescriptionGiven,FollowUpRequired,ReadmissionFlag,VisitDurationMinutes,VisitID
0,1,6,NaN,Takapuna,Coastal,Ear Infection,Viral Infection,2022-03-18,Outpatient,92.99,Yes,Insurance,No,No,No,208,332
1,1,2,2.0,Takapuna,Coastal,Asthma,Bronchitis,2022-04-14,Inpatient,2974.58,Yes,Insurance,Yes,Yes,Yes,51,460
2,1,9,9.0,Takapuna,Coastal,Ear Infection,Influenza,2022-06-03,Inpatient,4403.89,Yes,Insurance,Yes,Yes,No,96,702
3,1,7,NaN,Takapuna,Coastal,Ear Infection,Ear Infection,2022-07-14,Inpatient,2216.95,Yes,Insurance,Yes,No,No,104,976
4,1,7,7.0,Takapuna,Coastal,Ear Infection,Ear Infection,2022-09-22,Inpatient,2055.49,Yes,Insurance,Yes,Yes,No,82,1429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,350,14,14.0,Pukekohe,Inland,Hay Fever,Allergic Rhinitis,2024-09-06,Outpatient,94.01,No,Self-pay,Yes,Yes,No,228,8976
11996,350,23,NaN,Pukekohe,Inland,Hay Fever,Allergic Rhinitis,2024-10-26,Outpatient,179.09,Yes,Insurance,Yes,No,No,206,10032
11997,350,21,14.0,Pukekohe,Inland,Hay Fever,Asthma Flare-up,2024-11-18,Outpatient,172.68,Yes,Insurance,Yes,Yes,Yes,35,10628
11998,350,25,NaN,Pukekohe,Inland,Heat Exhaustion,Eczema,2024-12-07,Outpatient,161.01,Yes,Insurance,No,No,No,30,11171


In [61]:
df_aucklandvisits_to_sql['FollowUpDoctorID'] = df_aucklandvisits_to_sql['FollowUpDoctorID'].fillna(-1).astype(int)


In [64]:
# changed NaN in FollowUpDoctorID as -1

In [62]:
insert_visits(cursor,df_aucklandvisits_to_sql)

In [63]:
print("aucklandpatients.csv,aucklnaddoctors.csv & aucklandvisits.csv successfully pushed to SSMS")

aucklandpatients.csv,aucklnaddoctors.csv & aucklandvisits.csv successfully pushed to SSMS
